**Step Model**

Environment 
Python 3.6.8


**Step Imports**

In [1]:
!{sys.executable} -m pip install numpy opencv-python
!{sys.executable} -m pip install torch==1.8.2 torchvision==0.9.2 torchaudio===0.8.2 --extra-index-url https://download.pytorch.org/whl/lts/1.8/cu111
!{sys.executable} -m pip install ipywidgets
from pydoc import describe
from colorama import Style
from ipyfilechooser import FileChooser
import os
import ipywidgets as widgets
from IPython.display import HTML
import cv2
import numpy as np
import glob



Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/lts/1.8/cu111


**Step Inference**


In [2]:
# Step Inference
preTrainedModelPicker = FileChooser(os.getcwd())
preTrainedModelPicker.filter_pattern = "*.pth"
preTrainedModelPicker.title = "Select pre-trained model"
preTrainedModelPicker.default_path = os.path.join(os.getcwd(),"pretrained")


fcSelectFrameFolder = FileChooser(os.getcwd())
fcSelectFrameFolder.show_only_dirs = True
fcSelectFrameFolder.title = "Select video frame folder to run inference on:"
fcSelectFrameFolder.default_path = os.path.join(os.getcwd(),'datasets/demo/frames')
fcSelectFrameResultFolder = FileChooser(os.getcwd())
fcSelectFrameResultFolder.show_only_dirs = True
fcSelectFrameResultFolder.title = "Select folder to save inference results to:"
fcSelectFrameResultFolder.default_path = os.path.join(os.getcwd(),'datasets/demo/results')
selectVideoFrameFolder = widgets.HBox([widgets.Label("Save inference result to:"),fcSelectFrameFolder])
confirmBtn2 = widgets.Button(description="Confirm")
resetBtn2 = widgets.Button(description="Reset")
buttonsHbox2 = widgets.HBox([confirmBtn2,resetBtn2])
inferrenceOutput = widgets.Output()
vbox2 = widgets.VBox([preTrainedModelPicker,fcSelectFrameFolder, fcSelectFrameResultFolder,buttonsHbox2, inferrenceOutput])
def on_confirm_clicked2(b):
    with inferrenceOutput:
        print("Running inference with pre-trained model: ", os.path.normpath( preTrainedModelPicker.selected))
        print("Running inference on video frame folder: ", fcSelectFrameFolder.selected_path)
        print("Saving results to: ", fcSelectFrameResultFolder.selected_path)
        %run demo.py {os.path.join(fcSelectFrameFolder.selected_path,fcSelectFrameFolder.selected_filename)} {os.path.join(fcSelectFrameResultFolder.selected_path,fcSelectFrameResultFolder.selected_filename)} {os.path.normpath( preTrainedModelPicker.selected)}
def on_reset_clicked2(b):
    inferrenceOutput.clear_output()
    fcSelectFrameFolder.reset()
    fcSelectFrameResultFolder.reset()
confirmBtn2.on_click(on_confirm_clicked2)
resetBtn2.on_click(on_reset_clicked2)
display(vbox2)

**Step Frames Extraction**

In [3]:

fcSelectVideo = FileChooser(os.getcwd())
fcSelectVideo.title = "Select video folder to run frame extraction on:"
fcSelectVideo.default_path = os.path.join(os.getcwd(),'datasets/input/')
fcResults = FileChooser(os.getcwd())
fcResults.show_only_dirs = True
fcResults.title = "Select folder to save frame extraction results to:"
fcResults.default_path = os.path.join(os.getcwd(),'datasets/ava/')


confirmBtn = widgets.Button(description="Confirm")
resetBtn = widgets.Button(description="Reset")
buttonsHbox = widgets.HBox([confirmBtn,resetBtn])


output = widgets.Output()
vbox = widgets.VBox([fcSelectVideo,fcResults,buttonsHbox, output])
def on_confirm_clicked(b):
    with output:
        print("Running inference on video : ", os.path.join(fcSelectVideo.selected_path,fcSelectVideo.selected_filename))
        print("Saving results to: ", fcResults.selected_path)
        %run ./scripts/extract_clips2.py {os.path.join(fcSelectVideo.selected_path,fcSelectVideo.selected_filename)} {os.path.join(fcResults.selected_path,fcResults.selected_filename)}
        # %run ./scripts/extract_clips.py 
def on_reset_clicked(b):
    output.clear_output()
    fcSelectVideo.reset()
    fcResults.reset()
confirmBtn.on_click(on_confirm_clicked)
resetBtn.on_click(on_reset_clicked)




display(vbox)



**Step Training(Generate Label)**

In [4]:
# Step training generating label
trainingCsv = FileChooser(os.getcwd())
trainingCsv.filter_pattern = "*.csv"
trainingCsv.title = "Select training CSV file"
trainingCsv.default_path = os.path.join(os.getcwd(),"datasets/ava/label/")
valuationCsv = FileChooser(os.getcwd())
valuationCsv.filter_pattern = "*.csv"
valuationCsv.title = "Select valuation CSV file"
valuationCsv.default_path = os.path.join(os.getcwd(),"datasets/ava/label/")
trainingConfirmBtn = widgets.Button(description="Confirm")
trainingResetBtn = widgets.Button(description="Reset")
trainingButtonsHbox = widgets.HBox([trainingConfirmBtn,trainingResetBtn])
trainingOutput = widgets.Output()
trainingVbox = widgets.VBox([trainingCsv,valuationCsv,trainingButtonsHbox, trainingOutput])
def on_training_confirm_clicked(b):
    with trainingOutput:
        print("Generate label with csv file: ", os.path.normpath( trainingCsv.selected))
        %run ./scripts/generate_label.py {os.path.normpath( trainingCsv.selected)}
        %run ./scripts/generate_label.py {os.path.normpath( valuationCsv.selected)}
        print("done generating label")
def on_training_reset_clicked(b):
    trainingOutput.clear_output()
    trainingCsv.reset()
    valuationCsv.reset()
trainingConfirmBtn.on_click(on_training_confirm_clicked)
trainingResetBtn.on_click(on_training_reset_clicked)
display(trainingVbox)

**Step Training**

In [5]:
# Step Training Model
from cProfile import label
data_root="datasets/ava/"
save_root="datasets/ava/cache/"
pretrain_path="pretrained/ava_cls.pth"

name="STEP"
base_net="i3d"
det_net="two_branch"
resume_path="Auto"

T=3
max_iter=3    # index starts from 1
iterative_mode="temporal"
anchor_mode="1"
temporal_mode="predict"
pool_mode="align"
pool_size=7

# training schedule
num_workers=4
max_epochs=14
batch_size=1
optimizer="adam"
base_lr=7.5e-5
det_lr0=1.5e-4
det_lr=7.5e-4
save_step=11465
print_step=500
scheduler="cosine"
milestones="-1"
warmup_iters=1000

# losses
dropout=0.3
fc_dim=256
lambda_reg=5
lambda_neighbor=1
cls_thresh="0.2,0.35,0.5"
reg_thresh="0.2,0.35,0.5"
max_pos_num=5
neg_ratio=2
NUM_SAMPLE=-1
topk=300
evaluate_topk=300

# data augmentation / normalization
scale_norm=2    # for i3d
do_flip="True"
do_crop="True"
do_photometric="True"
do_erase="True"
freeze_affine="True"
freeze_stats="True"
outputTrain = widgets.Output()
dataFileFolderPath = FileChooser(os.getcwd())
dataFileFolderPath.title = "Select data file folder"
dataFileFolderPath.default_path = os.path.join(os.getcwd(),"datasets/ava/")
dataFileFolderPath.show_only_dirs = True
saveFileFolderPath = FileChooser(os.getcwd())
saveFileFolderPath.title = "Select save file folder"
saveFileFolderPath.default_path = os.path.join(os.getcwd(),"datasets/ava/cache/")
saveFileFolderPath.show_only_dirs = True
pretrainFilePath = FileChooser(os.getcwd())
pretrainFilePath.title = "Select pretrain file"
pretrainFilePath.default_path = os.path.join(os.getcwd(),"pretrained/")
pretrainFilePath.filter_pattern = "*.pth"
epochLabel = widgets.Label("Epoch")
epochInput = widgets.IntText();
epochInput.value = 14
batchLabel = widgets.Label("Batch")
batchInput = widgets.IntText();
batchInput.value = 1
workerLabel = widgets.Label("Worker")
workerInput = widgets.IntText();
workerInput.value = 4
epochHbox = widgets.HBox([epochLabel,epochInput])
batchHbox = widgets.HBox([batchLabel,batchInput])
workerHbox = widgets.HBox([workerLabel,workerInput])
startButton = widgets.Button(description="Start")
startButton.style.button_color = 'lightgreen'
startButton.style.font_weight = 'bold'
def startOnClick(b):
    with outputTrain:
        %run train.py --data_root $dataFileFolderPath.selected --save_root $saveFileFolderPath.selected \
        --name $name --pretrain_path $pretrainFilePath.selected --resume_path $resume_path \
        --base_net $base_net --det_net $det_net --max_iter $max_iter --T $T \
        --iterative_mode $iterative_mode --anchor_mode $anchor_mode --anchor_mode $anchor_mode --temporal_mode $temporal_mode \
        --pool_mode $pool_mode --pool_size $pool_size --save_step $save_step --topk $topk --evaluate_topk $evaluate_topk \
        --num_workers $workerInput.value --max_epochs $epochInput.value --batch_size $batchInput.value --print_step $print_step \
        --optimizer $optimizer --base_lr $base_lr --det_lr $det_lr --det_lr0 $det_lr0 --milestones $milestones \
        --scale_norm $scale_norm --do_flip $do_flip --do_crop $do_crop --do_photometric $do_photometric --do_erase $do_erase \
        --fc_dim $fc_dim --dropout $dropout --NUM_SAMPLE $NUM_SAMPLE --scheduler $scheduler --warmup_iters $warmup_iters \
        --cls_thresh $cls_thresh --reg_thresh $reg_thresh --max_pos_num $max_pos_num --neg_ratio $neg_ratio \
        --freeze_affine $freeze_affine --freeze_stats $freeze_stats --lambda_reg $lambda_reg --lambda_neighbor $lambda_neighbor
startButton.on_click(startOnClick)
display(dataFileFolderPath, saveFileFolderPath, pretrainFilePath, epochHbox, batchHbox, workerHbox, startButton, outputTrain)


FileChooser(path='c:\Users\user\Documents\GitHub\STEP3\datasets\ava', filename='', title='Select data file fol…

FileChooser(path='c:\Users\user\Documents\GitHub\STEP3\datasets\ava\cache', filename='', title='Select save fi…

FileChooser(path='c:\Users\user\Documents\GitHub\STEP3\pretrained', filename='', title='Select pretrain file',…

Button(description='Start', style=ButtonStyle(button_color='lightgreen', font_weight='bold'))

Output()

**Step Testing**


In [6]:
pretrainedTestFs = FileChooser(os.getcwd())
pretrainedTestFs.title = "Select pretrained file"
pretrainedTestFs.default_path = os.path.join(os.getcwd(),"pretrained/")
pretrainedTestFs.filter_pattern = "*.pth"
testOutput = widgets.Output()
testButton = widgets.Button(description="Test")
testButton.style.button_color = 'lightgreen'
testButton.style.font_weight = 'bold'
def testOnClick(b):
    with testOutput:
        %run test.py {pretrainedTestFs.selected}
testButton.on_click(testOnClick)
    
display(pretrainedTestFs, testButton, testOutput)

FileChooser(path='c:\Users\user\Documents\GitHub\STEP3\pretrained', filename='', title='Select pretrained file…

Button(description='Test', style=ButtonStyle(button_color='lightgreen', font_weight='bold'))

Output()

For reference

In [ ]:
data_root="datasets/ava/"
save_root="datasets/ava/cache/"
pretrain_path="pretrained/ava_cls.pth"

name="STEP"
base_net="i3d"
det_net="two_branch"
resume_path="Auto"

T=3
max_iter=3    # index starts from 1
iterative_mode="temporal"
anchor_mode="1"
temporal_mode="predict"
pool_mode="align"
pool_size=7

# training schedule
num_workers=4
max_epochs=14
batch_size=1
optimizer="adam"
base_lr=7.5e-5
det_lr0=1.5e-4
det_lr=7.5e-4
save_step=11465
print_step=500
scheduler="cosine"
milestones="-1"
warmup_iters=1000

# losses
dropout=0.3
fc_dim=256
lambda_reg=5
lambda_neighbor=1
cls_thresh="0.2,0.35,0.5"
reg_thresh="0.2,0.35,0.5"
max_pos_num=5
neg_ratio=2
NUM_SAMPLE=-1
topk=300
evaluate_topk=300

# data augmentation / normalization
scale_norm=2    # for i3d
do_flip="True"
do_crop="True"
do_photometric="True"
do_erase="True"
freeze_affine="True"
freeze_stats="True"


%run train.py --data_root $data_root --save_root $save_root \
--name $name --pretrain_path $pretrain_path --resume_path $resume_path \
    --base_net $base_net --det_net $det_net --max_iter $max_iter --T $T \
    --iterative_mode $iterative_mode --anchor_mode $anchor_mode --anchor_mode $anchor_mode --temporal_mode $temporal_mode \
    --pool_mode $pool_mode --pool_size $pool_size --save_step $save_step --topk $topk --evaluate_topk $evaluate_topk \
    --num_workers $num_workers --max_epochs $max_epochs --batch_size $batch_size --print_step $print_step \
    --optimizer $optimizer --base_lr $base_lr --det_lr $det_lr --det_lr0 $det_lr0 --milestones $milestones \
    --scale_norm $scale_norm --do_flip $do_flip --do_crop $do_crop --do_photometric $do_photometric --do_erase $do_erase \
    --fc_dim $fc_dim --dropout $dropout --NUM_SAMPLE $NUM_SAMPLE --scheduler $scheduler --warmup_iters $warmup_iters \
    --cls_thresh $cls_thresh --reg_thresh $reg_thresh --max_pos_num $max_pos_num --neg_ratio $neg_ratio \
    --freeze_affine $freeze_affine --freeze_stats $freeze_stats --lambda_reg $lambda_reg --lambda_neighbor $lambda_neighbor 

In [ ]:
%run test.py

In [ ]:
%run ./scripts/generate_label.py "datasets/ava/label/ava_train_v2.1.csv"
%run ./scripts/generate_label.py "datasets/ava/label/ava_val_v2.1.csv"